In [2]:
!pip install DeepXDE==0.10

gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

  Using cached https://files.pythonhosted.org/packages/42/a1/dfa3a829bc691a04e9da510586acdb69cce8d0ecff4984c7b003e70408f5/DeepXDE-0.10.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d3/eb/d0e658465c029feb7083139d9ead51000742e88b1fb7f1504e19e1b4ce6e/scikit_learn-0.24.2-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/fe/e2/20b693b5cc699f8116075379d57c4628502f58f96aa759209b77be047882/tensorflow-2.6.2-cp36-cp36m-manylinux2010_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/04/09/bcbe4b907c6d987ea66416178475c6049c9ab7b56135c3725b2e01e3c9e8/SALib-1.4.5-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/7c/91/d3ba0401e62d7e42816bc7d97b82d19c95c164b3e149a87c0a1c026a735e/joblib-1.1.1-py2.py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/61/cf/6e354304bcb9c6413c4e02a747b600061c21d38ba51e7e544ac7bc66aecc/threadpoolctl-3.1.0-py3-none-any.whl
  Using cached https://

In [3]:
import sys
import os
import csv
#dir_path = os.path.dirname(os.path.realpath(__file__))
#sys.path.append(dir_path)
dir_path = '/content/drive/MyDrive/Annie'
#sys.path.append(dir_path)
from sklearn.model_selection import train_test_split
from deepxde.backend import tf
import matplotlib.pyplot as plt
import argparse
import numpy as np
import deepxde as dde # version 0.11 or higher
#from generate_plot import plot_1D  # should be changed for the new one
#from generate_plots_2d import plot_2D
#from generate_plots_1d_MV import plot_1D
import utils


Using backend: tensorflow.compat.v1

2023-06-04 11:25:08.161943: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-04 11:25:08.490733: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-04 11:25:08.490792: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-04 11:25:10.601114: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object 

Instructions for updating:
non-resource variables are not supported in the long term


2023-06-04 11:25:13.640736: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-06-04 11:25:13.640883: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (cx3-15-29.cx3.hpc.ic.ac.uk): /proc/driver/nvidia/version does not exist


In [4]:
input_2d = 3 # network input size (2D)
num_hidden_layers_2d = 4 # number of hidden layers for NN (2D)
hidden_layer_size_2d = 32 # size of each hidden layers (2D)
output_2d = 3 # network output size (2D)
output_heter = 3 # network output size for heterogeneity case (2D)

In [5]:
## Training Parameters
num_domain = 20000 # number of training points within the domain
num_boundary = 1000 # number of training boundary condition points on the geometry boundary
num_test = 1000 # number of testing points within the domain
num_initial = 98 # number of points to test the initial conditions
MAX_MODEL_INIT = 16 # maximum number of times allowed to initialize the model
MAX_LOSS = 4 # upper limit to the initialized loss
epochs = 100000 #60000 # number of epochs for training
lr =  0.00005 # learning rate
noise = 0.1 # noise factor
test_size = 0.1 # precentage of testing data 

In [6]:
dim = 2
noise_introduced = False
inverse_activated = True
inverse_string = 'fix_tauo'
inverse = ['taud','taur','tausi']  #[inverse_string]
#model_folder_name = '/content'
animation = False
heter = False
w_used = False #data

In [7]:
dynamics = utils.system_dynamics()

In [8]:
params = dynamics.params_to_inverse(inverse)

Estimatig taud
Estimatig taur
Estimatig tausi


In [9]:
file_name =  'U.mat'
observe_x, u = dynamics.generate_data(file_name, dim)

In [10]:
train_mask = np.zeros(len(u), dtype=bool)
nlocs = len(np.unique(observe_x[:,0]))   #number of unique locations with observations
for i in range(nlocs):
    train_mask[i::10] = True

u_test = u[train_mask]
u_train = u[~train_mask]

observe_test = observe_x[train_mask]
observe_train = observe_x[~train_mask]


In [11]:
geomtime = dynamics.geometry_time(dim)

In [12]:
bc = dynamics.BC_func(dim, geomtime)

In [13]:
ic_u, ic_v, ic_w = dynamics.IC_func(observe_train, u_train, geomtime)

In [14]:
observe_u = dde.PointSetBC(observe_train, u_train, component=0)  # component says which component it is
input_data = [bc, ic_u, ic_v, ic_w, observe_u]

In [15]:
pde = dynamics.pde_2D


In [16]:
net = dde.maps.FNN([input_2d] + [hidden_layer_size_2d] * num_hidden_layers_2d + [output_2d], "tanh", "Glorot uniform")
pde_data = dde.data.TimePDE(geomtime, pde, input_data,
                            num_domain = num_domain,
                            num_boundary=num_boundary,
                            num_initial = num_initial,
                            anchors=observe_train,
                            num_test=num_test)
model = dde.Model(pde_data, net)

In [17]:
print("flag 1")
model.compile("adam", lr=lr)
print("flag 2")

flag 1
Compiling model...
Building feed-forward neural network...
'build' took 0.201582 s


/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/deepxde/nn/tensorflow_compat_v1/fnn.py:114: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  return tf.layers.dense(


2023-06-04 11:25:38.411694: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


'compile' took 2.061415 s

flag 2


In [19]:
losshistory, _ = model.train(epochs=1)
num_itertions = len(losshistory.loss_train)
initial_loss = max(losshistory.loss_train[num_itertions - 1])
num_init = 0
while initial_loss>MAX_LOSS or np.isnan(initial_loss).any() or np.isinf(initial_loss).any():  # added checking for inf values
    num_init += 1
    model = dde.Model(pde_data, net)
    model.compile("adam", lr=lr)
        #model.compile("adam", lr=lr, loss_weights=loss_weights)
    losshistory, _ = model.train(epochs=1)
    initial_loss = max(losshistory.loss_train[0])
    if num_init > MAX_MODEL_INIT:
        raise ValueError('Model initialization phase exceeded the allowed limit')

print("flag 3")

Initializing variables...


2023-06-04 11:26:44.960996: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_4/bias/Adam_1/Assign' id:1902 op device:{requested: '', assigned: ''} def:{{{node dense_4/bias/Adam_1/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@dense_4/bias"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](dense_4/bias/Adam_1, dense_4/bias/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training model...



InvalidArgumentError: Graph execution error:

Detected at node 'sub_11' defined at (most recent call last):
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio

In [21]:
out_path = 'train'

In [22]:
losshistory, train_state = model.train(epochs=epochs, model_save_path = out_path, display_every=1000)

Initializing variables...


2023-06-04 11:28:19.208776: W tensorflow/c/c_api.cc:291] Operation '{name:'dense_4/bias/Adam_1/Assign' id:1902 op device:{requested: '', assigned: ''} def:{{{node dense_4/bias/Adam_1/Assign}} = Assign[T=DT_FLOAT, _class=["loc:@dense_4/bias"], _has_manual_control_dependencies=true, use_locking=true, validate_shape=true](dense_4/bias/Adam_1, dense_4/bias/Adam_1/Initializer/zeros)}}' was changed by setting attribute after it was run by a session. This mutation will have no effect, and will trigger an error in the future. Either don't modify nodes after running them or create a new session.


Training model...



InvalidArgumentError: Graph execution error:

Detected at node 'sub_11' defined at (most recent call last):
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
      app.start()
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/rds/general/user/ag619/home/anaconda3/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio